In [1]:
import ipyleaflet as ipyl
import ipywidgets as ipyw
import json
import patatmo

In [2]:
# your patatmo connect developer credentials
credentials = json.load(open("C:\\users\\annefou\\Documents\\.netatmo.credentials"))

# create an api client
client = patatmo.api.client.NetatmoClient()
# tell the client's authentication your credentials
client.authentication.credentials = credentials
# optionally give the authentication a temporary file.
# The tokens are then stored there for later reuse,
# e.g. next time you invoke this script.
# This saves time because no new tokens have to be requested.
# New tokens are then only requested if the old ones expire.
client.authentication.tmpfile = "temp_auth.json"


In [3]:

# lat/lon outline of Hamburg/Germany
hamburg_region = {
    "lat_ne" : 53.7499,
    "lat_sw" : 53.3809,
    "lon_ne" : 10.3471,
    "lon_sw" : 9.7085,
}

# issue the API request
hamburg = client.Getpublicdata(region = hamburg_region)
# convert the response to a pandas.DataFrame
print(hamburg.dataframe)

<bound method GetpublicdataResponse.dataframe of patatmo.api.responsetypes.GetpublicdataResponse(
response = {'body': [{'_id': '70:ee:50:03:da:4c', 'place': {'location': [10.166938, 53.523264], 'altitude': 30, 'timezone': 'Europe/Berlin'}, 'mark': 15, 'measures': {'02:00:00:03:df:42': {'res': {'1513939992': [6.1, 96]}, 'type': ['temperature', 'humidity']}, '05:00:00:00:3d:cc': {'rain_60min': 0, 'rain_24h': 0.101, 'rain_live': 0, 'rain_timeutc': 1513939992}, '70:ee:50:03:da:4c': {'res': {'1513940003': [1032.3]}, 'type': ['pressure']}}, 'modules': ['02:00:00:03:df:42', '05:00:00:00:3d:cc'], 'module_types': {'02:00:00:03:df:42': 'NAModule1', '05:00:00:00:3d:cc': 'NAModule3'}}, {'_id': '70:ee:50:01:47:34', 'place': {'location': [10.16587, 53.50993], 'altitude': 24, 'timezone': 'Europe/Berlin'}, 'mark': 12, 'measures': {'02:00:00:01:3a:d0': {'res': {'1513940265': [7.2, 91]}, 'type': ['temperature', 'humidity']}, '06:00:00:00:4d:da': {'wind_strength': 2, 'wind_angle': 230, 'gust_strength': 4

In [16]:
print(hamburg.dataframe())

                     altitude  humidity                 id   latitude  \
id                                                                      
70:ee:50:03:da:4c   30.000000      95.0  70:ee:50:03:da:4c  53.523264   
70:ee:50:01:47:34   24.000000      91.0  70:ee:50:01:47:34  53.509930   
70:ee:50:03:de:82   25.000000      95.0  70:ee:50:03:de:82  53.522404   
70:ee:50:17:bd:96         NaN       NaN  70:ee:50:17:bd:96  53.530789   
70:ee:50:03:bc:2c   15.000000     100.0  70:ee:50:03:bc:2c  53.530948   
70:ee:50:03:72:28   30.000000      94.0  70:ee:50:03:72:28  53.545382   
70:ee:50:14:42:1c   24.000000       NaN  70:ee:50:14:42:1c  53.569867   
70:ee:50:01:3c:f6   26.000000      91.0  70:ee:50:01:3c:f6  53.581100   
70:ee:50:22:e6:0a         NaN      95.0  70:ee:50:22:e6:0a  53.574758   
70:ee:50:1f:72:fc         NaN      96.0  70:ee:50:1f:72:fc  53.605328   
70:ee:50:13:6a:64   18.000000      88.0  70:ee:50:13:6a:64  53.583755   
70:ee:50:27:89:04         NaN     100.0  70:ee:50:2

In [22]:
hamburg.dataframe()['temperature']

id
70:ee:50:03:da:4c     6.1
70:ee:50:01:47:34     7.2
70:ee:50:03:de:82     6.1
70:ee:50:17:bd:96     NaN
70:ee:50:03:bc:2c     5.8
70:ee:50:03:72:28     7.0
70:ee:50:14:42:1c     NaN
70:ee:50:01:3c:f6     7.2
70:ee:50:22:e6:0a     6.3
70:ee:50:1f:72:fc     5.8
70:ee:50:13:6a:64     8.0
70:ee:50:27:89:04     6.5
70:ee:50:13:52:52     NaN
70:ee:50:12:bf:82     6.0
70:ee:50:05:39:30     NaN
70:ee:50:04:fc:56     7.5
70:ee:50:03:b4:60     5.8
70:ee:50:03:75:4e     6.5
70:ee:50:13:65:16     6.6
70:ee:50:04:f4:38     5.9
70:ee:50:00:87:0a     NaN
70:ee:50:03:da:b4     6.2
70:ee:50:13:5d:98     6.5
70:ee:50:1b:0f:70     6.1
70:ee:50:15:e5:50     8.4
70:ee:50:12:f4:6a     6.9
70:ee:50:17:77:b0     7.5
70:ee:50:03:9d:aa     7.2
70:ee:50:13:64:06     6.7
70:ee:50:12:95:b8     6.5
                     ... 
70:ee:50:02:05:84     5.5
70:ee:50:01:cc:50     5.8
70:ee:50:13:7b:be     7.1
70:ee:50:04:fe:3a     5.9
70:ee:50:27:1a:d4     5.3
70:ee:50:04:fd:0e     6.1
70:ee:50:19:9b:50     NaN
70:ee:50:

In [ ]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [ ]:
cols = ['temperature', 'altitude', 'humidity']
geojson = df_to_geojson(hamburg.dataframe(), cols)

In [21]:
# Map and label widgets
map = ipyl.Map(center=[53.88, 27.45], zoom=4)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))


layer = ipyl.GeoJSON(data=geojson)

map.add_layer(layer)


ipyw.VBox([map, label])

VBox(children=(Map(center=[53.88, 27.45], layers=(TileLayer(options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size']), GeoJSON(data={'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'temperature': 6.1, 'altitude': 30.0, 'humidity': 96.0}, 'geometry': {'type': 'Point', 'coordinates': [10.166938, 53.523264]}}, {'type': 'Feature', 'properties': {'temperature': 7.2, 'altitude': 24.0, 'humidity': 91.0}, 'geometry': {'type': 'Point', 'coordinates': [10.16587, 53.50993]}}, {'type': 'Feature', 'properties': {'temperature': 6.1, 'altitude': 25.0, 'humidity': 94.0}, 'geometry': {'type': 'Point', 'coordinates': [10.15244102478, 53.522404488088]}}, {'type': 'Feature', 'properties': {'temperature': nan, 'altitude': nan, 'humidity': nan}, 'geometry': {'type': 'Point', 'coordinates': [10.127101, 53.530789]}}, {'type': 'Feature', 'properties': {'temperature': 5.8, 'altitude': 15.0, 'humidity': 100.0}, 'geometry': {'type': 'Point', 'coordinates': [10.134062, 53.530948]}}, {'type': 'Feature', 'properties': {'temperature': 7.0, 'altitude': 30.0, 'humidity': 94.0}, 'geometry': {'type': 'Point', 'coordinates': [10.159016, 53.545382]}}, {'type': 'Feature', 'properties': {'temperature': nan, 'altitude': 24.0, 'humidity': nan}, 'geometry': {'type': 'Point', 'coordinates': [10.1554532, 53.5698669]}}, {'type': 'Feature', 'properties': {'temperature': 7.2, 'altitude': 26.0, 'humidity': 91.0}, 'geometry': {'type': 'Point', 'coordinates': [10.1485, 53.5811]}}, {'type': 'Feature', 'properties': {'temperature': 6.3, 'altitude': nan, 'humidity': 95.0}, 'geometry': {'type': 'Point', 'coordinates': [10.126336, 53.574758]}}, {'type': 'Feature', 'properties': {'temperature': 5.8, 'altitude': nan, 'humidity': 96.0}, 'geometry': {'type': 'Point', 'coordinates': [10.130407, 53.605328]}}, {'type': 'Feature', 'properties': {'temperature': 8.0, 'altitude': 18.0, 'humidity': 88.0}, 'geometry': {'type': 'Point', 'coordinates': [10.130053, 53.583755]}}, {'type': 'Feature', 'properties': {'temperature': 6.5, 'altitude': nan, 'humidity': 99.0}, 'geometry': {'type': 'Point', 'coordinates': [10.162313, 53.606906]}}, {'type': 'Feature', 'properties': {'temperature': nan, 'altitude': 38.0, 'humidity': nan}, 'geometry': {'type': 'Point', 'coordinates': [10.13860842214, 53.623712696185]}}, {'type': 'Feature', 'properties': {'temperature': 6.0, 'altitude': 38.0, 'humidity': 100.0}, 'geometry': {'type': 'Point', 'coordinates': [10.141229337759, 53.621220693695]}}, {'type': 'Feature', 'properties': {'temperature': nan, 'altitude': 37.0, 'humidity': nan}, 'geometry': {'type': 'Point', 'coordinates': [10.138314, 53.634115]}}, {'type': 'Feature', 'properties': {'temperature': 7.5, 'altitude': 31.0, 'humidity': 90.0}, 'geometry': {'type': 'Point', 'coordinates': [10.131676, 53.64696]}}, {'type': 'Feature', 'properties': {'temperature': 5.8, 'altitude': 18.547319, 'humidity': 100.0}, 'geometry': {'type': 'Point', 'coordinates': [10.146439, 53.648383]}}, {'type': 'Feature', 'properties': {'temperature': 6.5, 'altitude': 26.0, 'humidity': 98.0}, 'geometry': {'type': 'Point', 'coordinates': [10.125911515216, 53.639906281655]}}, {'type': 'Feature', 'properties': {'temperature': 6.6, 'altitude': 27.0, 'humidity': 94.0}, 'geometry': {'type': 'Point', 'coordinates': [10.1358438, 53.6906536]}}, {'type': 'Feature', 'properties': {'temperature': 5.9, 'altitude': 36.0, 'humidity': 95.0}, 'geometry': {'type': 'Point', 'coordinates': [10.141585880426, 53.672133481224]}}, {'type': 'Feature', 'properties': {'temperature': nan, 'altitude': 26.0, 'humidity': nan}, 'geometry': {'type': 'Point', 'coordinates': [10.129657, 53.690201]}}, {'type': 'Feature', 'properties': {'temperature': 6.2, 'altitude': 27.0, 'humidity': 96.0}, 'geometry': {'type': 'Point', 'coordinates': [10.14114, 53.696098]}}, {'type': 'Feature', 'properties': {'temperature': 6.5, 'altitude': 29.0, 'humidity': 100.0}, 'geometry': {'type': 'Point', 'coordinates': [10.147759521084